In [11]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# if nltk data is not present
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ifte/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# Path to the data txt file on disk.
data_path = "../Dataset/English_for_Today.txt"

In [12]:
# Here mecab tokenizer needed to tokenize the input data 
# Vectorize the data.
input_texts = []
target_texts = []

with open(data_path, 'r', encoding='utf-8') as f:
    #lines = f.read().split('\n')
    lines = f.read()

#print(lines)   

# Here mecab tokenizer needed to tokenize the input data 
wordsList = nltk.word_tokenize(lines)
#print(wordsList)

In [15]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('Col%d(C-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('Col%d(C)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('Col%d(C+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [24]:
npwordsList = np.asarray(wordsList) 
npwordsList.shape

(56002,)

In [ ]:
encoded = LabelEncoder()
e = encoded.fit_transform(npwordsList)

In [23]:
e.shape

(56002,)

In [26]:
# we can also use padding technique here to make the length of number of token even number
# shaping according to desired pattern samples, Timestep, Predicted output length 
d = e[:-2].reshape(npwordsList[:len(npwordsList)-2].shape[0]//10,10)
d.shape

(5600, 10)

In [27]:
n_hours = 3
n_features = 5
n_ahead = 10

# frame as supervised learning
reframed = series_to_supervised(d, n_hours, n_features)
reframed

,Col1(C-3),Col2(C-3),Col3(C-3),Col4(C-3),Col5(C-3),Col6(C-3),Col7(C-3),Col8(C-3),Col9(C-3),Col10(C-3),...,Col1(C+4),Col2(C+4),Col3(C+4),Col4(C+4),Col5(C+4),Col6(C+4),Col7(C+4),Col8(C+4),Col9(C+4),Col10(C+4)
3,1684.0,2747.0,6587.0,1531.0,884.0,56.0,1983.0,749.0,2446.0,2205.0,...,2182.0,2039.0,985.0,1065.0,1703.0,549.0,1577.0,345.0,1982.0,931.0
4,1983.0,4013.0,6587.0,2222.0,6076.0,346.0,975.0,1039.0,1955.0,847.0,...,873.0,1475.0,1204.0,1297.0,865.0,863.0,1344.0,1808.0,1386.0,878.0
5,532.0,2157.0,1730.0,1833.0,1437.0,84.0,2180.0,1151.0,1133.0,1788.0,...,1928.0,691.0,1931.0,565.0,1224.0,1699.0,1306.0,928.0,1531.0,885.0
6,1941.0,1788.0,1411.0,1437.0,564.0,1747.0,1523.0,1831.0,986.0,1391.0,...,56.0,1983.0,749.0,1078.0,1091.0,946.0,3969.0,2222.0,7105.0,343.0
7,1794.0,1191.0,1517.0,792.0,56.0,1951.0,689.0,71.0,892.0,1704.0,...,2747.0,6587.0,1134.0,5194.0,708.0,84.0,1691.0,2747.0,549.0,1682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,6603.0,6599.0,6993.0,6595.0,553.0,683.0,665.0,549.0,1748.0,2324.0,...,2989.0,84.0,325.0,2156.0,6763.0,3330.0,72.0,5216.0,2216.0,1387.0
5592,6587.0,3995.0,5338.0,5194.0,6587.0,6381.0,71.0,6593.0,7095.0,2377.0,...,2359.0,6587.0,5263.0,2216.0,602.0,6598.0,7063.0,87.0,5414.0,7113.0
5593,6646.0,6598.0,5671.0,84.0,91.0,1413.0,2205.0,2862.0,7044.0,6763.0,...,4229.0,4750.0,4422.0,6587.0,2217.0,5670.0,84.0,2171.0,2764.0,2336.0
5594,2989.0,84.0,854.0,91.0,3969.0,1387.0,2359.0,854.0,325.0,3969.0,...,6847.0,7119.0,5288.0,7063.0,2359.0,5414.0,6646.0,3325.0,6590.0,84.0


In [ ]:
#!/usr/bin/env python

npwordsList = np.asarray(wordsList) 




# In[31]:


#Index(['Col1(C-3)', 'Col2(C-3)', 'Col3(C-3)', 'Col4(C-3)', 'Col5(C-3)',
#       'Col6(C-3)', 'Col7(C-3)', 'Col8(C-3)', 'Col9(C-3)', 'Col10(C-3)',
#       'Col1(C-2)', 'Col2(C-2)', 'Col3(C-2)', 'Col4(C-2)', 'Col5(C-2)',
#       'Col6(C-2)', 'Col7(C-2)', 'Col8(C-2)', 'Col9(C-2)', 'Col10(C-2)',
#       'Col1(C-1)', 'Col2(C-1)', 'Col3(C-1)', 'Col4(C-1)', 'Col5(C-1)',
#       'Col6(C-1)', 'Col7(C-1)', 'Col8(C-1)', 'Col9(C-1)', 'Col10(C-1)',
#       'Col1(C)', 'Col2(C)', 'Col3(C)', 'Col4(C)', 'Col5(C)', 'Col6(C)',
#       'Col7(C)', 'Col8(C)', 'Col9(C)', 'Col10(C)', 'Col1(C+1)', 'Col2(C+1)',
#       'Col3(C+1)', 'Col4(C+1)', 'Col5(C+1)', 'Col6(C+1)', 'Col7(C+1)',
#       'Col8(C+1)', 'Col9(C+1)', 'Col10(C+1)', 'Col1(C+2)', 'Col2(C+2)',
#       'Col3(C+2)', 'Col4(C+2)', 'Col5(C+2)', 'Col6(C+2)', 'Col7(C+2)',
#       'Col8(C+2)', 'Col9(C+2)', 'Col10(C+2)', 'Col1(C+3)', 'Col2(C+3)',
#       'Col3(C+3)', 'Col4(C+3)', 'Col5(C+3)', 'Col6(C+3)', 'Col7(C+3)',
#       'Col8(C+3)', 'Col9(C+3)', 'Col10(C+3)', 'Col1(C+4)', 'Col2(C+4)',
#       'Col3(C+4)', 'Col4(C+4)', 'Col5(C+4)', 'Col6(C+4)', 'Col7(C+4)',
#       'Col8(C+4)', 'Col9(C+4)', 'Col10(C+4)'],
#      dtype='object')
        




st = (n_hours+1)*n_ahead + 1  

# drop columns we don't want to predict
deletedcol =    list(reframed.columns)[st           :  st+n_ahead-1] +                 list(reframed.columns)[st+n_ahead   :  st+n_ahead*2-1] +                 list(reframed.columns)[st+n_ahead*2 :  st+n_ahead*3-1] +                 list(reframed.columns)[st+n_ahead*3 : ] 
#print("deleted column",deletedcol)

reframed.drop(deletedcol, axis=1, inplace=True)
#print(reframed.head())

#reframed.to_csv(r'outNum.csv')


# In[52]:


# decoding to see the formation


# split into train and test sets
values = reframed.values
train_size = int(len(values) * 0.8)
test_size = len(values) - train_size
train, test = values[0:train_size,:], values[train_size:,:]

input_col = (n_hours+1)*n_ahead

## split into input and outputs
train_X, train_y = train[:, :input_col], train[:, -4:]
test_X, test_y = test[:, :input_col], test[:, -4:]
#print(train_X, train_X.shape, train_y, train_y.shape)


# In[53]:


print(train_X.shape, test_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# In[57]:


# design network
model = Sequential()
model.add(LSTM(160, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(4))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=40, validation_data=(test_X, test_y), verbose=2, shuffle=False)


# In[62]:


# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()


# In[63]:


# make a prediction
yhat = model.predict(test_X)
yhat


# In[66]:


yhat = yhat.reshape(-1)
print(yhat.shape)
encoded.inverse_transform()


# In[48]:



#test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
# invert scaling for forecast
#inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
#inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

